In [1]:
# закомментировать - Command + ?

# Собираем Dataset

In [4]:
# Подключаем библиотеки

# Для парсинга файла html
from bs4 import BeautifulSoup 
import re
import os
import fnmatch
import numpy as np

# Создаем список, элементы которого будут представлять текст статьи на опредленную тему. 
collection = []

# Всего областей 5: a) биология, b) информационные технологии, c) история, d) менеджмент, 
# e) архитектура и строительство
# Создадим список, в котором каждому номеру документа соответствует область
field = []


# Читаем все файлы из папки dataset
for filename in os.listdir("dataset"):
    with open(os.path.join("dataset", filename), "rb") as f:
        contents = f.read().decode(errors='replace')
        soup = BeautifulSoup(contents, 'html.parser')
        
        a = soup.find_all('p', style = re.compile("[\D+]?text-align: justify;"))
        s = ''
        
        for i in range(len(a)):
        
# Некоторые атрибуты в файле имели пустые значения и при записи почему-то записывались как \xa0, поэтому
# пришлось проводить проверку на то, чтобы a[i].text != '\xa0' и только потом записывать в общую строку
            if a[i].text != '\xa0':
                s += ' ' + re.sub('[a-zA-Z0-9%-.,:—;"!№/\[\]\\\'?»«–’”“℃]','',a[i].text.lower())
         
    
# При парсинге не получалось удалить некоторые объекты, пришлось ручками.
    s = s.replace('\u200b','')
    s = s.replace('\xad', '')
    s = s.replace('\xa0', '')
    collection.append(s)
    
    if fnmatch.fnmatch(filename, 'a*.html'):
        field.append('Биология')
    elif fnmatch.fnmatch(filename, 'b*.html'):
        field.append('Информационные технологии')
    elif fnmatch.fnmatch(filename, 'c*.html'):
        field.append('История')
    elif fnmatch.fnmatch(filename, 'd*.html'):
        field.append('Менеджмент')
    elif fnmatch.fnmatch(filename, 'e*.html'):
        field.append('Архитектура')
    else: field.append('1')
    
collection.remove(collection[14])
field.remove(field[14])

# field

# Мощность датасета и пример одного элемента 
# print(len(collection))
# print(collection[108])

# Предобработка данных

Данные уже :
1. приведены к нижнему регистру
2. отчищены от пунктуационных знаков
3. отчищены от спец.знаков

В этом разделе проведем лемматизацию

In [5]:
data = [a.split() for a in collection]
# data

In [6]:
# Здесь я решила удалить слова, короче 4-ех символов. Но в этом случае IDF для всех остальных почему-то 0.0
# data = []
# for i in range(len(collection_split)):
#     data.append(list(filter(lambda x: len(x) > 4, collection_split[1])))
# data

In [13]:
# !pip3 install pymorphy2

  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=423da816e91843a4667b484c487086cdbdf871d673036152fd748a74b12585ff
  Stored in directory: /Users/natalaantonenko/Library/Caches/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [14]:
# !pip3 install pymorphy2-dicts

  Using cached pymorphy2_dicts-2.4.393442.3710985-py2.py3-none-any.whl (7.1 MB)


In [15]:
# !pip3 install DAWG-Python

In [16]:
# Напишем функцию для лемматизации текста и выведем коллекцию статей снова
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def lemmat(data):
    res_data = []
    for words in data:
        res_line = []
        for word in words:
            res_line.append(morph.parse(word)[0].normal_form)
        res_data.append(' '.join(res_line)) 
    return res_data

collection = lemmat(data)

In [17]:
collection[0]

'в период пандемия площадка электронный коммерция получить широкий распространение в связь с ограниченность работа объект торговля всё больший количество человек склоняться к покупка в режим онлайн но на увеличение количество маркетплейс повлиять не только пандемия но возрастание темп жизнь и развитие технология в статья производиться анализ и классификация существующий интернетрынка и применять технология знание типология маркетплейс позволить сформировать суждение о тот в какой именно категория создание платформа интернет торговля быть наиболее привлекательный для клиент ключевой слово интернетмагазин вебсайт торговый площадка товар бизнес интернетторговый площадка это вебсайт или приложение который упрощать совершение покупка из множество различный источник оператор торговый площадка не владеть никакой товар он бизнес заключаться в тот чтобы предоставить пользователь товар другой человек и облегчить транзакция и хороший пример онлайнрынок они продавать всё и весь поскольку такой тор

In [18]:
data = [a.split() for a in collection] 
#data

# Term Frequency (TF)

Посчитаем частоту слова в документе. Для этого сначала напишем фукцию.

In [9]:
import collections

def tf(word, text):
    w_count = 0
    for i in text:
        if i == word:
            w_count += 1
    return w_count / float(len(text))

In [10]:
print(tf('в',data[10]))

0.03502824858757062


In [11]:
print(tf('площадка',data[0]))

0.005430566330488751


# Inverse Document Frequency (IDF)

Посчитаем обратную частоту документа. Для этого сначала напишем фукцию.

In [12]:
import math

def idf(word, corpus):
        return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

Примеры работы для слов "площадка" и "в"

In [13]:
print(idf('площадка', data))

1.0374264979406236


In [14]:
print(idf('в', data))

0.0


# TF-IDF

In [15]:
# На вход подается слово, документ и вся коллекция (оба в виде списков слов)
# На выходе - tf-idf значение для слова в документе

def tf_idf(word, text, collection):
    return tf(word, text) * idf(word, collection)

In [16]:
tf_idf('площадка', data[0], data)

0.0056338134100732086

In [17]:
tf_idf('в', data[0], data)

0.0

# BM25

Посчитаем BM25

In [18]:
avgdl = sum(len(text) for text in data) / len(data)
N = len(data)

def bm25(word, text, k = 1.25, b = 0.75):
    freq = text.count(word)
    tf = (freq * (k + 1)) / (freq + k * 
                             (1 - b + b * len(text) / avgdl))
    # Колличество документов, содержащих слово
    N_q = sum([1 for doc in data if word in doc]) 
    idf = np.log(((N - N_q + 0.5) / (N_q + 0.5)) + 1)
    return round(tf * idf, 5)

In [19]:
#collection[5]

In [20]:
bm25('площадка', data[0])

4.06864

In [21]:
bm25('интеллект', data[5])

6.96879

In [22]:
bm25('интеллект', data[10])

0.0

# Векторизация текста (по TF-IDF и BM25)

In [23]:
# Создаем словарь всех слов 

set_data = []
for i in range(len(data)):
    for word in data[i]:
        set_data.append(word)
set_data = set(set_data)

# set_data

In [24]:
# Векторизуем текст по TF-IDF

def vec_tfidf(text, collection):
    vec_tfidf = []
    for word in set_data:
        vec_tfidf.append(tf_idf(word, text, collection))
    return vec_tfidf

In [25]:
# Векторизуем текст по BM25

def vec_bm25(text):
    vec_bm25 = []
    for word in set_data:
        vec_bm25.append(bm25(word, text))
    return vec_bm25

In [26]:
# Пример работы функций:

# vec_tfidf(data[2], data)
# vec_bm25(data[2])

# Близость документов

In [27]:
# Напишем функцию, высчитывающую косинусное расстояние

def cos(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / np.linalg.norm(vec1) / np.linalg.norm(vec2)

# Пример с поиском по запросу

In [28]:
# Введем запрос и преобразуем его
search = 'Виды растений животных'
search = lemmat([search.lower().split()])[0].split()
search

['вид', 'растение', 'животное']

In [29]:
# Построим вектора запроса

vec_search = vec_tfidf(search, data)
vec_search_bm25 = vec_bm25(search)

In [30]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar = []
for i in range(len(data)):
    scalar.append(cos(vec_tfidf(data[i], data), vec_search))

In [31]:
# len(scalar)

In [32]:
# Список индексов подходящих документов

res_index = [scalar.index(x) for x in scalar if x != 0.0]
# res_index

In [33]:
# Список соответствующих областей для проверки качественной работы программы

# for i in res_index:
#     print(field[i])

In [34]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar_bm25 = []
for i in range(len(data)):
    scalar_bm25.append(cos(vec_bm25(data[i]), vec_search_bm25))

In [35]:
# scalar_bm25

In [36]:
res_index_bm25 = [scalar_bm25.index(x) for x in scalar_bm25 if x != 0.0]
# res_index_bm25

In [37]:
# [collection[i] for i in res_index_bm25]

# Ранжирование

Для нашего запроса будем выводить область статьи и саму статью. Ранжировать будем по убыванию близости ветора запроса и вектора статьи. 
Проделаем процедуру только для scalar_bm25

In [38]:
# Создание словаря с номерами статей и соответствующими косинусными расстояниями. 
# Словарь сортируется по возрастанию косинусного расстояния. А выводит в обратном порядке, для наглядности.

rang_index = {}

for i in range(len(scalar_bm25)):
    rang_index[i] = scalar_bm25[i]

list_rang_index = list(rang_index.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

82 : 0.09216818756380038
103 : 0.07345910375678946
89 : 0.06856384792815508
25 : 0.06578544535497653
72 : 0.05933188346906328
73 : 0.05770521929646847
41 : 0.051227168484913994
14 : 0.049698399565836694
105 : 0.046778245661627325
49 : 0.04653926052674324
50 : 0.04469382645427097
96 : 0.041689123745517406
7 : 0.0405632778553857
28 : 0.03007883952746028
56 : 0.029091108114527805
43 : 0.02837653449002781
40 : 0.027220414410943132
84 : 0.006136537625558745
77 : 0.004722195878622466
52 : 0.004722195878622466
31 : 0.004472440775125026
39 : 0.004451299924072514
75 : 0.00429966349237328
80 : 0.004195365571542861
55 : 0.004195365571542861
23 : 0.004190838420940793
4 : 0.004190838420940793
57 : 0.004084086565666489
10 : 0.0038443929557030318
46 : 0.0037138736709677335
104 : 0.0037024268040546614
70 : 0.003575644788228151
94 : 0.0035676352217556754
68 : 0.0035676352217556754
32 : 0.003562822943432044
13 : 0.003304762168288524
0 : 0.0032007730404067
2 : 0.0031645849572670168
93 : 0.003126072543033

In [48]:
# Выведем первые 10 подходящих статей. Я вывожу области, из которых взяты статьи, чтобы предварительно показать
# качество работы модели. Вывод самих статей закомментирован. Чтобы посмотреть - раскомментируйте 7ю строку.
res_top20 = []

for index in reversed(list_rang_index[-20:]):
    if index[1] != 0.0:
#         print(field[index[0]])
        print(index[0])
        res_top20.append(index[0])
#         print(collection[index[0]])
#         print('\n')

82
103
89
25
72
73
41
14
105
49
50
96
7
28
56
43
40
84
77
52


# Предварительная оценка качества 

По запросу "Программное обеспечение" выводятся почти все статьи из области "Информационные тенологии", а так же статьи из других областей, включающие данное (или близкое к данному*) словосочетание или слова поотдельности.

\* Близкое, в данном смысле, обозначает все слова с другими склонениями (Н-р, слово "программная" тоже подходит). Это реализовано благодаря предварительной лемматизации такста.

# Подготовка датасета для оценки качества

Оценка качества модели производится следующим образом:
1. Строится новый список документов по запросу, содержащему синонимы к главным словам
2. Строится список из первых 20 подходящих статей 
3. Список сравнивается со списком первых 20 статей по изходному запросу
4. Считается доля входящих статей первого списка во второй.

In [40]:
# Введем запрос и преобразуем его
search_acr = 'Вид тип разновидность семейство растений трава цветы и животных скот зверь'
search_acr = lemmat([search_acr.lower().split()])[0].split()
search_acr

['вид',
 'тип',
 'разновидность',
 'семейство',
 'растение',
 'трава',
 'цветок',
 'и',
 'животное',
 'скот',
 'зверь']

In [51]:
vec_search_acr_bm25 = vec_bm25(search_acr)

In [52]:
# Построим список косинусного расстояния между вектором запроса и векторами статей

scalar_acr_bm25 = []
for i in range(len(data)):
    scalar_acr_bm25.append(cos(vec_bm25(data[i]), vec_search_acr_bm25))

In [53]:
# Создание словаря с номерами статей и соответствующими косинусными расстояниями. 
# Словарь сортируется по возрастанию косинусного расстояния. А выводит в обратном порядке, для наглядности.

rang_index = {}

for i in range(len(scalar_acr_bm25)):
    rang_index[i] = scalar_acr_bm25[i]

list_rang_index = list(rang_index.items())
list_rang_index.sort(key = lambda i: i[1])

for index in reversed(list_rang_index):
    print(index[0], ':', index[1])

82 : 0.11883774996510967
105 : 0.10753640672500413
50 : 0.09122907490443881
25 : 0.07510994882802138
49 : 0.057707120874950904
73 : 0.05584034676571202
14 : 0.047302239543167285
5 : 0.04067297988868171
43 : 0.03710783104438478
103 : 0.03416432987979654
89 : 0.02815675087631137
36 : 0.02709732074139464
70 : 0.026216176348824553
72 : 0.02436556183241292
32 : 0.022340993374426254
41 : 0.021037374907584606
96 : 0.017120406273030013
7 : 0.01665830984220277
28 : 0.013475833517906707
56 : 0.01194709840152043
40 : 0.011178795095401216
23 : 0.008236358583802098
4 : 0.008236358583802098
26 : 0.007717900124395898
99 : 0.007352448875907159
13 : 0.006514608884328137
93 : 0.006201010482422412
79 : 0.00616160309267397
0 : 0.006056978115389238
94 : 0.005665113451404955
68 : 0.005665113451404955
33 : 0.004985119797785618
6 : 0.004985119797785618
21 : 0.004972617020895845
34 : 0.004867847016326531
16 : 0.004867847016326531
35 : 0.004650373674756906
62 : 0.004493500133965691
67 : 0.004486817371535643
42 

In [54]:
res_acr = []

for index in reversed(list_rang_index[-20:]):
    if index[1] != 0.0:
        res_acr.append(index[0])

res_acr

[82,
 105,
 50,
 25,
 49,
 73,
 14,
 5,
 43,
 103,
 89,
 36,
 70,
 72,
 32,
 41,
 96,
 7,
 28,
 56]

# Оценка качества модели

In [58]:
k = sum([1.0 for i in res_top20 if i in res_acr])
k/20

0.8

Вывод: из топ 20-ти верных (по моему способу) статей - 16 распознала модель. Это неплохо